In [10]:
from __future__ import print_function

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,TimeDistributed
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import layers,models
import h5py
from keras.models import Model
from keras.layers import Layer
import cv2

from keras.callbacks import ModelCheckpoint


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import heapq
import warnings
warnings.filterwarnings('ignore')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10075308940086761895
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7505887651246425420
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4067257079579608019
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5109579776
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1724205501372336380
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [20]:

class ScaleLayer(Layer):

    def __init__(self,scale, **kwargs):

        self.scale=scale;
        
        super(ScaleLayer, self).__init__(**kwargs)


    def build(self, input_shape):
        super(ScaleLayer, self).build(input_shape)

       
    def call(self, inputs):

        output=tf.image.resize(inputs,[self.scale,self.scale],method=tf.image.ResizeMethod.BICUBIC)

        output = tf.keras.backend.concatenate((output, output,output),axis=-1) 


        print(output.shape)

        return output

       

    def compute_output_shape(self, input_shape):
        return (input_shape[0],self.scale,self.scale,3)



def vgg16_head(img_input):

    pooling = 'No'

    include_top = False;



    x = TimeDistributed(ScaleLayer(224))(img_input)

    x = TimeDistributed(layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1'))(x)
    x = TimeDistributed(layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2'))(x)
    x = TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))(x)

    # Block 2
    x = TimeDistributed(layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1'))(x)
    x = TimeDistributed(layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2'))(x)
    x = TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))(x)

    # Block 3
    x = TimeDistributed(layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1'))(x)
    x = TimeDistributed(layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2'))(x)
    x = TimeDistributed(layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3'))(x)
    x = TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))(x)

    # Block 4
    x = TimeDistributed(layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1'))(x)
    x = TimeDistributed(layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2'))(x)
    x = TimeDistributed(layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3'))(x)
    x = TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))(x)

    # Block 5
    x = TimeDistributed(layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1'))(x)
    x = TimeDistributed(layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2'))(x)
    x = TimeDistributed(layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3'))(x)
    x = TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))(x)

    if include_top:
        # Classification block
        x = layers.Flatten(name='flatten')(x)
        x = layers.Dense(4096, activation='relu', name='fc1')(x)
        x = layers.Dense(4096, activation='relu', name='fc2')(x)
        x = layers.Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)
        elif pooling =='No':
            x = x;
    return x

In [21]:

class AttentionLayer(Layer):

    def __init__(self, **kwargs):

        
        super(AttentionLayer, self).__init__(**kwargs)


    def build(self, input_shape):
        super(AttentionLayer, self).build(input_shape)


        self.input_shape_r = input_shape;
        print('self.input_shape_r')
        print(self.input_shape_r)

        L_size = 256;

        self.w = self.add_weight(name='w', 
                              shape=[L_size,1],
                              initializer='uniform',
                              trainable=True)
        self.V = self.add_weight(name='V', 
                      shape=[input_shape[2],L_size],
                      initializer='uniform',
                      trainable=True)

        self.U = self.add_weight(name='U', 
                      shape=[input_shape[2],L_size],
                      initializer='uniform',
                      trainable=True)

        

        self.bias_w = self.add_weight(shape=[1,1],
                              initializer='uniform',
                              name='bias_w')
        self.bias_V = self.add_weight(shape=[1,L_size],
                                      initializer='uniform',
                                      name='bias_V')
        self.bias_U = self.add_weight(shape=[1,L_size],
                                      initializer='uniform',
                                      name='bias_U')
        

  
       
    def call(self, inputs):


        inputs_r = K.expand_dims(inputs,axis=-2)


        print('inputs')
        print(inputs_r.shape)


        dot_products_V = K.dot(inputs_r,self.V)
        dot_products_U = K.dot(inputs_r,self.U)

        dot_products_V += self.bias_V;
        dot_products_U += self.bias_U;


        print('dot product')
        print(dot_products_V.shape)

        tanh_V = K.tanh(dot_products_V);
        sigmoid_U = K.sigmoid(dot_products_U);

        print('tanh')
        print(tanh_V.shape)

        comb_UV = tanh_V*sigmoid_U;

        print('combUV')
        print(comb_UV.shape)

        attention_weights = K.dot(comb_UV,self.w);

        attention_weights +=self.bias_w;

        attention_weights = K.exp(attention_weights);


        print('attention_weights')
        print(attention_weights.shape)


        attention_weights /= K.cast(K.sum(attention_weights,
                                  axis=1,
                                  keepdims=True) + K.epsilon(),
                            K.floatx());
        print('attention_weights sum')
        print(attention_weights.shape)

        #attention_weights = K.expand_dims(attention_weights);
        attention_weights= K.squeeze(attention_weights,axis = -1);
        print('attention_weights last')
        print(attention_weights.shape)

        weighted_output = inputs * attention_weights;

        self.Save =  attention_weights;


        output = K.sum(weighted_output, axis=1)

        print('output')
        print(output.shape)

        return output


       

    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[2])
    
    
class FCHeadNet:
  @staticmethod
  def build(baseModel, classes, D):
    # initialize the head model that will be placed on top of
    # the base, then add a FC layer
    headModel = baseModel.output
    headModel = TimeDistributed(layers.Flatten(name='flatten'))(headModel)
    headModel = TimeDistributed(layers.Dense(D, activation='relu', name='fc1'))(headModel)
    headModel = TimeDistributed(layers.Dense(D, activation='relu', name='fc2'))(headModel)
    headModel = AttentionLayer(name='attentionlayer')(headModel)
    headModel = layers.Dense(classes, activation='softmax', name='predictions')(headModel)
    # add a softmax layer
    #headModel = layers.Dense(classes, activation="softmax")(headModel)
    #headModel = layers.Dense(classes, activation='softmax', name='predictions2')(headModel)

    # return the model
    return headModel


In [17]:
import random
batch_size = 1
num_classes = 3


# input image dimensions
img_rows, img_cols = 128, 128

# the data, split between train and test sets

#(x_train, y_train), (x_test, y_test) = mnist.load_data()


p='/home/peppermint/Data_Sci/Attention_ML/1160_many.h5';

db=h5py.File(p);


X_stk, y_stk = db['images'],db['labels'];

X =np.zeros((116,10,128,128,3))
y = np.zeros((116,))

for ij in range(116):
    X[ij,...] = X_stk[ij*10:(ij+1)*10,...]
    y[ij,...] = y_stk[(ij*10+(ij+1)*10)//2]

print(X.shape)
print(y.shape)


total_size = X.shape[0];
index_random = np.arange(total_size)
i = int(X.shape[0] * 3/4)

random.shuffle(index_random)

train_index = list(index_random[:i]) ;
test_index = list(index_random[i:]) ;
train_index.sort()
test_index.sort()


x_train, x_test, y_train, y_test = X[train_index],X[test_index],y[train_index],y[test_index];

print(x_train.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train_gray_zero = x_train[...,0]
x_train_gray_first = x_train[...,1]
x_train_gray_second = x_train[...,2]

x_train_gray_zero = x_train_gray_zero[...,None]
x_train_gray_first = x_train_gray_first[...,None]
x_train_gray_second = x_train_gray_second[...,None]


x_test_gray_zero = x_test[...,0]
x_test_gray_first = x_test[...,1]
x_test_gray_second = x_test[...,2]


x_test_gray_zero = x_test_gray_zero[...,None]
x_test_gray_first = x_test_gray_first[...,None]
x_test_gray_second = x_test_gray_second[...,None]

(116, 10, 128, 128, 3)
(116,)
(87, 10, 128, 128, 3)


In [18]:
x_test_gray_zero.shape

(29, 10, 128, 128, 1)

In [22]:
img_rows, img_cols = 128, 128

pooling = 'No'
epochs = 50

num_classes = 3
num_sequence = 10;
include_top = False;

input_shape =  [num_sequence,img_rows, img_cols, 1];
img_input = layers.Input(shape=input_shape)
inputs = img_input


x = vgg16_head(inputs);

# Create model.
base_model = models.Model(inputs, x, name='vgg16')

weights ='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5';

base_model.load_weights(weights)


head_model = FCHeadNet.build(base_model, num_classes, 256)

model = Model(inputs=base_model.input, outputs=head_model)

Dont_Want_to_train_all = True;


if (Dont_Want_to_train_all):

  for layer in base_model.layers:
    layer.trainable = False

mcp_save = ModelCheckpoint('weights_lung_gray_zero.h5', save_best_only=True, monitor='val_loss', mode='min')


model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

model.fit(x_train_gray_zero, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_gray_zero, y_test),
         callbacks=[mcp_save])

(?, 224, 224, 3)
self.input_shape_r
(None, 10, 256)
inputs
(?, 10, 1, 256)
dot product
(?, 10, 1, 256)
tanh
(?, 10, 1, 256)
combUV
(?, 10, 1, 256)
attention_weights
(?, 10, 1, 1)
attention_weights sum
(?, 10, 1, 1)
attention_weights last
(?, 10, 1)
output
(?, 256)
Train on 87 samples, validate on 29 samples
Epoch 1/50
87/87 [==============================] - 11s 127ms/step - loss: 0.6564 - accuracy: 0.6628 - val_loss: 0.6078 - val_accuracy: 0.6437
Epoch 2/50
87/87 [==============================] - 8s 92ms/step - loss: 0.5230 - accuracy: 0.7548 - val_loss: 0.5258 - val_accuracy: 0.6897
Epoch 3/50
87/87 [==============================] - 8s 92ms/step - loss: 0.4706 - accuracy: 0.7625 - val_loss: 0.3969 - val_accuracy: 0.8276
Epoch 4/50
87/87 [==============================] - 8s 93ms/step - loss: 0.3842 - accuracy: 0.8046 - val_loss: 0.4638 - val_accuracy: 0.7471
Epoch 5/50
87/87 [==============================] - 8s 93ms/step - loss: 0.3794 - accuracy: 0.8391 - val_loss: 0.3458 - val_a

In [23]:
img_rows, img_cols = 128, 128

pooling = 'No'
epochs = 50

num_classes = 3
num_sequence = 10;
include_top = False;

input_shape =  [num_sequence,img_rows, img_cols, 1];
img_input = layers.Input(shape=input_shape)
inputs = img_input


x = vgg16_head(inputs);

# Create model.
base_model = models.Model(inputs, x, name='vgg16')

weights ='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5';

base_model.load_weights(weights)


head_model = FCHeadNet.build(base_model, num_classes, 256)

model = Model(inputs=base_model.input, outputs=head_model)

Dont_Want_to_train_all = True;


if (Dont_Want_to_train_all):

  for layer in base_model.layers:
    layer.trainable = False

mcp_save = ModelCheckpoint('weights_lung_gray_first.h5', save_best_only=True, monitor='val_loss', mode='min')


model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

model.fit(x_train_gray_first, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_gray_first, y_test),
         callbacks=[mcp_save])

(?, 224, 224, 3)
self.input_shape_r
(None, 10, 256)
inputs
(?, 10, 1, 256)
dot product
(?, 10, 1, 256)
tanh
(?, 10, 1, 256)
combUV
(?, 10, 1, 256)
attention_weights
(?, 10, 1, 1)
attention_weights sum
(?, 10, 1, 1)
attention_weights last
(?, 10, 1)
output
(?, 256)
Train on 87 samples, validate on 29 samples
Epoch 1/50
87/87 [==============================] - 9s 106ms/step - loss: 0.7084 - accuracy: 0.6322 - val_loss: 0.5959 - val_accuracy: 0.6667
Epoch 2/50
87/87 [==============================] - 9s 102ms/step - loss: 0.5664 - accuracy: 0.7050 - val_loss: 0.5093 - val_accuracy: 0.7356
Epoch 3/50
87/87 [==============================] - 9s 103ms/step - loss: 0.5038 - accuracy: 0.7625 - val_loss: 0.4922 - val_accuracy: 0.7356
Epoch 4/50
87/87 [==============================] - 9s 103ms/step - loss: 0.4861 - accuracy: 0.7625 - val_loss: 0.4799 - val_accuracy: 0.7701
Epoch 5/50
87/87 [==============================] - 9s 103ms/step - loss: 0.4074 - accuracy: 0.8084 - val_loss: 0.3999 - va

In [24]:
img_rows, img_cols = 128, 128

pooling = 'No'
epochs = 50

num_classes = 3
num_sequence = 10;
include_top = False;

input_shape =  [num_sequence,img_rows, img_cols, 1];
img_input = layers.Input(shape=input_shape)
inputs = img_input


x = vgg16_head(inputs);

# Create model.
base_model = models.Model(inputs, x, name='vgg16')

weights ='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5';

base_model.load_weights(weights)


head_model = FCHeadNet.build(base_model, num_classes, 256)

model = Model(inputs=base_model.input, outputs=head_model)

Dont_Want_to_train_all = True;


if (Dont_Want_to_train_all):

  for layer in base_model.layers:
    layer.trainable = False

mcp_save = ModelCheckpoint('weights_lung_gray_second.h5', save_best_only=True, monitor='val_loss', mode='min')


model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

model.fit(x_train_gray_second, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_gray_second, y_test),
         callbacks=[mcp_save])

(?, 224, 224, 3)
self.input_shape_r
(None, 10, 256)
inputs
(?, 10, 1, 256)
dot product
(?, 10, 1, 256)
tanh
(?, 10, 1, 256)
combUV
(?, 10, 1, 256)
attention_weights
(?, 10, 1, 1)
attention_weights sum
(?, 10, 1, 1)
attention_weights last
(?, 10, 1)
output
(?, 256)
Train on 87 samples, validate on 29 samples
Epoch 1/50
87/87 [==============================] - 9s 109ms/step - loss: 0.6787 - accuracy: 0.6245 - val_loss: 0.6161 - val_accuracy: 0.6322
Epoch 2/50
87/87 [==============================] - 9s 104ms/step - loss: 0.6145 - accuracy: 0.6743 - val_loss: 0.5678 - val_accuracy: 0.7011
Epoch 3/50
87/87 [==============================] - 9s 105ms/step - loss: 0.5482 - accuracy: 0.7126 - val_loss: 0.5236 - val_accuracy: 0.7816
Epoch 4/50
87/87 [==============================] - 9s 105ms/step - loss: 0.5385 - accuracy: 0.7165 - val_loss: 0.5786 - val_accuracy: 0.6667
Epoch 5/50
87/87 [==============================] - 9s 105ms/step - loss: 0.4757 - accuracy: 0.7739 - val_loss: 0.4704 - va

In [25]:
save_dir = "./"


weights_dir = save_dir + "weights_lung.h5"

In [26]:
model.save_weights(weights_dir)

In [27]:
model_dir_current = weights_dir


model.load_weights(model_dir_current)


In [28]:
Epoch 13/50
 - 9s 99ms/step - loss: 0.0360 - accuracy: 1.0000 - val_loss: 0.3433 - val_accuracy: 0.8851

SyntaxError: invalid syntax (<ipython-input-28-1a15acb6c744>, line 1)